In [1]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
file = open("Feature_mapping.csv", "w", encoding="utf8")
f = csv.writer(file)

# Feature columns + label for each Userid
f.writerow(["Userid", "Still Morning Weekday", "Still Afternoon Weekday", "Still Evening Weekday", "Still Night Weekday", 
            "Still Morning Weekend", "Still Afternoon Weekend", "Still Evening Weekend",  "Still Night Weekend"])

184

In [3]:
df = pd.read_csv("Activity_clean.csv")
df = df.sort_values(by=['Userid','senseStartTimeMillis'],ascending=[True,True])

# get the list of ordered ids
ids = df["Userid"].unique()
ids
# get the list of ordered surveyDates
# surveyDates = df.loc["Survey Date"].unique()


array([158455, 313449, 351278, 446875, 453897, 483491, 494149, 547832,
       738648, 773684, 873115, 931499, 953571])

In [4]:
Userid = []
StillMorningWeekday = []

In [5]:
def identifyTime(time,weekday):
    if weekday == 1:
        t = "06:00:00"
        morning = datetime.strptime(t, "%H:%M:%S").time()
        t = "22:00:00"
        night = datetime.strptime(t, "%H:%M:%S").time()
    else:
        t = "08:00:00"
        morning = datetime.strptime(t, "%H:%M:%S").time()
        t = "23:59:59"
        night = datetime.strptime(t, "%H:%M:%S").time()
    t = "12:00:00"
    afternoon = datetime.strptime(t, "%H:%M:%S").time()
    t = "17:00:00"
    evening = datetime.strptime(t, "%H:%M:%S").time()
    time = time[1:-2]
    time_obj = datetime.strptime(time, "%H:%M:%S").time()
    
    if weekday == 1:
        if time_obj < morning or time_obj >= night:
            return "night"
    else:
        if time_obj < morning:
            return "night"
    if time_obj >= morning and time_obj < afternoon:
        return "morning"
    if time_obj >= afternoon and time_obj < evening:
        return "afternoon"
    if time_obj >= evening and time_obj <= night:
        return "evening"

In [6]:
def determineWeekday(date):
    time = date
    time = time[2:-1]
    time = time.replace("T", " ")
    time = time.replace("-", "/")
    datetime_object = datetime.strptime(time, '%Y/%m/%d %H:%M:%S')
    dayOfTheWeek = datetime_object.weekday()

    if dayOfTheWeek >= 0 and dayOfTheWeek <= 4:
        weekday = 1
    else:
        weekday = 0
    
    return weekday, dayOfTheWeek
    

In [16]:
def CalculateStill(de_date, index, threshold, first, last):
    thr = 0 
    toAdd = 0 
#     print("Losing it",de_date['Activity'][index],index)
#     previousStill = 0
    # check if the element before is "Still"
    if first == 0:
        previousStill = 0
#     if de_date['Activity'][index-1] == " 'Still'":
#         print("here", de_date['Activity'][index-1])
#             previousStill = 0
    else:
        if de_date['Activity'][index-1] != " 'Still'":
            previousStill = 0
        else:
            diff = de_date['senseStartTimeMillis'][index] - de_date['senseStartTimeMillis'][index-1]
#             print(diff,index)
            if diff > 3600000:
                previousStill = 0
            else:
                previousStill = 1
#             print(diff,de_date['Activity'][index-1], previousStill)
    
    # check if the element after is "Still"
    if last == len(de_date['Activity']) - 1:
        afterStill = 0
    else:
        if de_date['Activity'][index+1] != " 'Still'":
            afterStill = 0
        else:
            diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
            # make param
            if diff > 3600000:
                afterStill = 0
            else:
                afterStill = 1
    
    if afterStill == 0 and last != len(de_date['Activity']) - 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        if diff <= 3600000:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
            
    if afterStill == 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        toAdd = diff
        
    if last == len(de_date['Activity']) - 1:
        time = de_date['Final Date'][index] 
        time = time[2:-1]
        time = time.replace("-", "/")
        # make param
        time += " 12:00:00"
        convert = time.mktime(datetime.datetime.strptime(time,
                                             '%Y/%m/%d %H:%M:%S').timetuple())
        diff = convert - de_date['senseStartTimeMillis'][index]
        diff = datetime.strptime("12:00:00", "%H:%M:%S").time()
        if diff <= 3600000:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
        
    return toAdd, thr, previousStill, afterStill

In [17]:
# print(len(df["Activity"]))

In [23]:
for id in ids:
    # get survey dates for the current id
    surveydates = df.loc[df["Userid"]==id, 'Survey Date'].tolist()
    uniquesurveydates = np.array(surveydates)
    uniquesurveydates = np.unique(uniquesurveydates)
    surveydates = uniquesurveydates.tolist()

    # get a dataframe of the data corresponding to current id
    de_id = df.loc[(df["Userid"])==int(id)]


    # total time of being still for a survey date
    totalWeekdayMorningStill = 0
    toAdd = 0
    # calculate features for each survey date 
    for date in surveydates:
        # retrive dataframe that contains information for current survey date
        de_date = 0
        de_date = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date)]
        # unique dates
        uniq = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date), 'Final Date'].tolist()
        uniq = np.array(uniq)
        uniq = np.unique(uniq)
        uniq = uniq.tolist()
        
        
        first = 0
        last = 0
        
        # iterate through dataframe and calculate features        
        for index in de_date.index:
            # determine weekend or weekday
            weekday, dayOfTheWeek = determineWeekday(de_date["senseStartTime"][index])
            
            # determine part of the day 
            partOfDay = identifyTime(de_date["Time"][index],weekday)
            
            if de_date["Activity"][index] == " 'Still'":
                if (partOfDay == "morning" and weekday == 1):
                    timeStill, thr, previousStill, afterStill = CalculateStill(de_date, index, 1, first, last)
                    
                    # If it is a completely new bundle
                    if previousStill == 0:
                        # add bundle time to total time
                        totalWeekdayMorningStill += toAdd
                        # reset bundle time
                        toAdd  = 0
                      
                    # if thr has not been reached add to existing bundle 
                    if thr == 0:
                        toAdd += timeStill
                    # if thr has been reached 
                    else:
                        if toAdd < 3600000:
                            if timeStill > 3600000:
                                toAdd = 3600000
                            else:
                                toAdd += timeStill
                        else:
                        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
                            toAdd += 600000 
#                     if id == 158455 and de_date["Survey Date"][index] == '"2022-11-01"':
#                         print(de_date['senseStartTime'][index],id,toAdd)
                                   
                    if last == len(de_date["Activity"]) - 1:
                        totalWeekdayMorningStill += toAdd
#                     if afterStill == 0:
#                         totalWeekdayMorningStill += toAdd
#                         toAdd  = 0
#                 if (id == 931499):
#                     print()
            first = 1
            last += 1
#     print(len(uniq))
        avgWeekdayMorningStill = totalWeekdayMorningStill / len(uniq)
#         print(totalWeekdayMorningStill,id)
#         print(avgWeekdayMorningStill,id)
            


4114285.714285714 158455
7920000.0 158455
7200000.0 158455
9771428.57142857 158455
2608281.3333333335 313449
4408281.333333333 313449
8436187.142857144 313449
13290660.57142857 313449
17582085.14285714 313449
24986125.2 313449
19309852.85714286 313449
39527207.5 313449
26351471.666666668 313449
39527207.5 313449
0.0 351278
2057142.857142857 351278
5134761.714285715 446875
9307699.57142857 446875
14056954.0 446875
19087586.333333332 446875
17568775.0 446875
22225849.42857143 446875
30120983.333333332 446875
41607093.2 446875
1904021.4 453897
8905890.57142857 453897
13534462.0 453897
19939364.333333332 453897
31709046.5 453897
23539364.333333332 453897
25216642.85714286 453897
29774007.42857143 453897
5168417.285714285 483491
10246243.857142856 483491
13825453.285714285 483491
27078200.5 483491
19725560.85714286 483491
24885918.0 483491
0.0 494149
5657142.857142857 494149
12750000.0 494149
13400000.0 494149
3818818.1428571427 547832
8044235.571428572 547832
12739038.714285715 547832
1718

In [10]:
t = df.loc[(df["Userid"]==158455) & (df["Survey Date"]=='"2022-11-01"')]

t

,Userid,senseStartTimeMillis,senseStartTime,Activity,Survey Date,Final Date,Time
0,158455,1666743438337,'2022-10-26T00:17:18','Still',"""2022-11-01""","""2022-10-26""","""00:17:18'"""
1,158455,1666745976112,'2022-10-26T00:59:36','Unknown',"""2022-11-01""","""2022-10-26""","""00:59:36'"""
2,158455,1666746581283,'2022-10-26T01:09:41','Still',"""2022-11-01""","""2022-10-26""","""01:09:41'"""
3,158455,1666747255485,'2022-10-26T01:20:55','Still',"""2022-11-01""","""2022-10-26""","""01:20:55'"""
4,158455,1666747863494,'2022-10-26T01:31:03','Still',"""2022-11-01""","""2022-10-26""","""01:31:03'"""
...,...,...,...,...,...,...,...
276,158455,1667336846411,'2022-11-01T21:07:26','Tilting',"""2022-11-01""","""2022-11-01""","""21:07:26'"""
277,158455,1667337493977,'2022-11-01T21:18:13','Tilting',"""2022-11-01""","""2022-11-01""","""21:18:13'"""
278,158455,1667337494007,'2022-11-01T21:18:14','Still',"""2022-11-01""","""2022-11-01""","""21:18:14'"""
279,158455,1667338099695,'2022-11-01T21:28:19','Tilting',"""2022-11-01""","""2022-11-01""","""21:28:19'"""
